In [104]:
import pandas as pd
import requests as rq
import utils.constants as constants
import streamlit as st

url1 = constants.RDW_KENTEKEN_URL
url2 = constants.RDW_BRANDSTOF_API_URL
json_kenteken = rq.get(url1, 
headers={"X-App-Token": st.secrets['RDW_API_KEY']}, params={"$limit": 500_000}).json()

json_brandstof = rq.get(url2, 
headers={"X-App-Token": st.secrets['RDW_API_KEY']}, params={"$limit": 500_000}).json()

df1 = pd.DataFrame.from_dict(json_kenteken)
df2 = pd.DataFrame.from_dict(json_brandstof)

print(len(df1))

# df1['kenteken'] = df1['kenteken'].astype(str)
# df2['kenteken'] = df2['kenteken'].astype(str)

# df1.info()
# print(df1)
# print(df2)
df = pd.merge(df1, df2, on='kenteken')

print(len(df))

# response = rq.get(constants.OCM_API_URL, 
# headers={"X-API-Key": st.secrets['OCM_API_KEY']}).json()
# pd.DataFrame.from_dict(response).head(5)

MemoryError: 